In [26]:
#Importing necessary libraries
import pandas as pd
from sklearn import tree
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import TransformerMixin
import pycaret
from pycaret.classification import *
% matplotlib inline

UsageError: Line magic function `%` not found.


In [2]:
df_train = pd.read_csv("H:/Learnings/Data Science/Hackathon/Mobility Analytics/train_Wc8LBpr.csv")
df_test =  pd.read_csv("H:/Learnings/Data Science/Hackathon/Mobility Analytics/test_VsU9xXK.csv")

In [3]:
df_train.isnull().sum()

Trip_ID                            0
Trip_Distance                      0
Type_of_Cab                    20210
Customer_Since_Months           5920
Life_Style_Index               20193
Confidence_Life_Style_Index    20193
Destination_Type                   0
Customer_Rating                    0
Cancellation_Last_1Month           0
Var1                           71030
Var2                               0
Var3                               0
Gender                             0
Surge_Pricing_Type                 0
dtype: int64

In [4]:
df_test.isnull().sum()

Trip_ID                            0
Trip_Distance                      0
Type_of_Cab                    13158
Customer_Since_Months           3966
Life_Style_Index               13327
Confidence_Life_Style_Index    13327
Destination_Type                   0
Customer_Rating                    0
Cancellation_Last_1Month           0
Var1                           46789
Var2                               0
Var3                               0
Gender                             0
dtype: int64

In [5]:
df_train.shape

(131662, 14)

In [6]:
df_test.shape

(87395, 13)

In [7]:
#Let's drop the Var1 feature as 50% are having Null values
df_train.drop(['Var1'],1,inplace=True)
df_test.drop(['Var1'],1,inplace=True)

In [8]:
df_train.shape

(131662, 13)

In [9]:
df_test.shape

(87395, 12)

In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131662 entries, 0 to 131661
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Trip_ID                      131662 non-null  object 
 1   Trip_Distance                131662 non-null  float64
 2   Type_of_Cab                  111452 non-null  object 
 3   Customer_Since_Months        125742 non-null  float64
 4   Life_Style_Index             111469 non-null  float64
 5   Confidence_Life_Style_Index  111469 non-null  object 
 6   Destination_Type             131662 non-null  object 
 7   Customer_Rating              131662 non-null  float64
 8   Cancellation_Last_1Month     131662 non-null  int64  
 9   Var2                         131662 non-null  int64  
 10  Var3                         131662 non-null  int64  
 11  Gender                       131662 non-null  object 
 12  Surge_Pricing_Type           131662 non-null  int64  
dtyp

In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87395 entries, 0 to 87394
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Trip_ID                      87395 non-null  object 
 1   Trip_Distance                87395 non-null  float64
 2   Type_of_Cab                  74237 non-null  object 
 3   Customer_Since_Months        83429 non-null  float64
 4   Life_Style_Index             74068 non-null  float64
 5   Confidence_Life_Style_Index  74068 non-null  object 
 6   Destination_Type             87395 non-null  object 
 7   Customer_Rating              87395 non-null  float64
 8   Cancellation_Last_1Month     87395 non-null  int64  
 9   Var2                         87395 non-null  int64  
 10  Var3                         87395 non-null  int64  
 11  Gender                       87395 non-null  object 
dtypes: float64(4), int64(3), object(5)
memory usage: 8.0+ MB


In [12]:
df_train['Surge_Pricing_Type'].value_counts()

2    56728
3    47720
1    27214
Name: Surge_Pricing_Type, dtype: int64

In [13]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [14]:
#Imputing the Null values in Train and Test dataset
df_train['Type_of_Cab'] = DataFrameImputer().fit_transform(pd.DataFrame(df_train['Type_of_Cab']))
df_train['Customer_Since_Months'] = DataFrameImputer().fit_transform(pd.DataFrame(df_train['Customer_Since_Months']))
df_train['Life_Style_Index'] = DataFrameImputer().fit_transform(pd.DataFrame(df_train['Life_Style_Index']))
df_train['Confidence_Life_Style_Index'] = DataFrameImputer().fit_transform(pd.DataFrame(df_train['Confidence_Life_Style_Index']))
df_test['Type_of_Cab'] = DataFrameImputer().fit_transform(pd.DataFrame(df_test['Type_of_Cab']))
df_test['Customer_Since_Months'] = DataFrameImputer().fit_transform(pd.DataFrame(df_test['Customer_Since_Months']))
df_test['Life_Style_Index'] = DataFrameImputer().fit_transform(pd.DataFrame(df_test['Life_Style_Index']))
df_test['Confidence_Life_Style_Index'] = DataFrameImputer().fit_transform(pd.DataFrame(df_test['Confidence_Life_Style_Index']))

In [15]:
df_train.isnull().sum()

Trip_ID                        0
Trip_Distance                  0
Type_of_Cab                    0
Customer_Since_Months          0
Life_Style_Index               0
Confidence_Life_Style_Index    0
Destination_Type               0
Customer_Rating                0
Cancellation_Last_1Month       0
Var2                           0
Var3                           0
Gender                         0
Surge_Pricing_Type             0
dtype: int64

In [16]:
df_test.isnull().sum()

Trip_ID                        0
Trip_Distance                  0
Type_of_Cab                    0
Customer_Since_Months          0
Life_Style_Index               0
Confidence_Life_Style_Index    0
Destination_Type               0
Customer_Rating                0
Cancellation_Last_1Month       0
Var2                           0
Var3                           0
Gender                         0
dtype: int64

In [18]:
cat_cols = ['Type_of_Cab', 'Confidence_Life_Style_Index', 'Destination_Type', 'Gender']
dummy_df = pd.DataFrame()
for i in cat_cols:
    i_dummies_train = pd.get_dummies(df_train[i], prefix = i)
    i_dummies_test = pd.get_dummies(df_test[i], prefix = i)
    j_train = len(i_dummies_train.columns)
    j_test = len(i_dummies_test.columns)
    i_dummies_train.drop(i_dummies_train.columns[j_train-1], axis = 1, inplace = True)
    i_dummies_test.drop(i_dummies_test.columns[j_test-1], axis = 1, inplace = True)
    df_train = pd.concat([df_train, i_dummies_train], axis = 1)
    df_test = pd.concat([df_test, i_dummies_test], axis = 1)

In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131662 entries, 0 to 131661
Data columns (total 33 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Trip_ID                        131662 non-null  object 
 1   Trip_Distance                  131662 non-null  float64
 2   Type_of_Cab                    131662 non-null  object 
 3   Customer_Since_Months          131662 non-null  float64
 4   Life_Style_Index               131662 non-null  float64
 5   Confidence_Life_Style_Index    131662 non-null  object 
 6   Destination_Type               131662 non-null  object 
 7   Customer_Rating                131662 non-null  float64
 8   Cancellation_Last_1Month       131662 non-null  int64  
 9   Var2                           131662 non-null  int64  
 10  Var3                           131662 non-null  int64  
 11  Gender                         131662 non-null  object 
 12  Surge_Pricing_Type            

In [20]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87395 entries, 0 to 87394
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Trip_ID                        87395 non-null  object 
 1   Trip_Distance                  87395 non-null  float64
 2   Type_of_Cab                    87395 non-null  object 
 3   Customer_Since_Months          87395 non-null  float64
 4   Life_Style_Index               87395 non-null  float64
 5   Confidence_Life_Style_Index    87395 non-null  object 
 6   Destination_Type               87395 non-null  object 
 7   Customer_Rating                87395 non-null  float64
 8   Cancellation_Last_1Month       87395 non-null  int64  
 9   Var2                           87395 non-null  int64  
 10  Var3                           87395 non-null  int64  
 11  Gender                         87395 non-null  object 
 12  Type_of_Cab_A                  87395 non-null 

In [21]:
# Dropping original columns from train dataset
df_train.drop(['Type_of_Cab', 'Confidence_Life_Style_Index', 'Destination_Type', 'Gender'],axis = 1, inplace = True )
df_test.drop(['Type_of_Cab', 'Confidence_Life_Style_Index', 'Destination_Type', 'Gender'],axis = 1, inplace = True )

In [22]:
x_train = df_train.drop(['Surge_Pricing_Type','Trip_ID'],1)
y_train = df_train['Surge_Pricing_Type']
x_test= df_test.drop(['Trip_ID'],1)

In [28]:
# prepare configuration for cross validation test harness
seed = 123
# prepare models
models = []
models.append(('ADB', AdaBoostClassifier()))
models.append(('GB', GradientBoostingClassifier()))
models.append(('LGBM', LGBMClassifier()))
models.append(('CAT', CatBoostClassifier()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, shuffle =True, random_state=seed)
    cv_results = model_selection.cross_val_score(model, x_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

ADB: 0.676740 (0.003161)
GB: 0.687237 (0.001956)
LGBM: 0.690283 (0.001904)
0:	learn: 1.0804937	total: 81ms	remaining: 1m 20s
1:	learn: 1.0637716	total: 108ms	remaining: 54s
2:	learn: 1.0480778	total: 134ms	remaining: 44.6s
3:	learn: 1.0339284	total: 162ms	remaining: 40.4s
4:	learn: 1.0203803	total: 188ms	remaining: 37.5s
5:	learn: 1.0076897	total: 214ms	remaining: 35.5s
6:	learn: 0.9957843	total: 240ms	remaining: 34.1s
7:	learn: 0.9848029	total: 267ms	remaining: 33s
8:	learn: 0.9741687	total: 298ms	remaining: 32.8s
9:	learn: 0.9641039	total: 324ms	remaining: 32.1s
10:	learn: 0.9547309	total: 351ms	remaining: 31.5s
11:	learn: 0.9458185	total: 376ms	remaining: 31s
12:	learn: 0.9374205	total: 402ms	remaining: 30.5s
13:	learn: 0.9295072	total: 428ms	remaining: 30.1s
14:	learn: 0.9219821	total: 457ms	remaining: 30s
15:	learn: 0.9149103	total: 495ms	remaining: 30.5s
16:	learn: 0.9082428	total: 528ms	remaining: 30.5s
17:	learn: 0.9018839	total: 562ms	remaining: 30.7s
18:	learn: 0.8958316	tota

### Using CATBOOST

In [29]:
cb = CatBoostClassifier()
cb.fit(x_train,y_train)

0:	learn: 1.0804940	total: 30ms	remaining: 30s
1:	learn: 1.0636675	total: 56.7ms	remaining: 28.3s
2:	learn: 1.0480062	total: 83.2ms	remaining: 27.6s
3:	learn: 1.0338896	total: 112ms	remaining: 27.8s
4:	learn: 1.0204212	total: 139ms	remaining: 27.7s
5:	learn: 1.0076972	total: 166ms	remaining: 27.5s
6:	learn: 0.9956799	total: 194ms	remaining: 27.5s
7:	learn: 0.9844428	total: 221ms	remaining: 27.4s
8:	learn: 0.9738407	total: 250ms	remaining: 27.6s
9:	learn: 0.9638193	total: 280ms	remaining: 27.7s
10:	learn: 0.9544367	total: 307ms	remaining: 27.6s
11:	learn: 0.9455535	total: 333ms	remaining: 27.4s
12:	learn: 0.9371826	total: 358ms	remaining: 27.2s
13:	learn: 0.9292251	total: 385ms	remaining: 27.1s
14:	learn: 0.9217352	total: 412ms	remaining: 27s
15:	learn: 0.9146583	total: 442ms	remaining: 27.2s
16:	learn: 0.9079898	total: 467ms	remaining: 27s
17:	learn: 0.9016241	total: 494ms	remaining: 27s
18:	learn: 0.8955369	total: 521ms	remaining: 26.9s
19:	learn: 0.8897816	total: 549ms	remaining: 26.

In [30]:
y_pred = cb.predict(x_test)

In [31]:
y_pred

array([[1.],
       [2.],
       [2.],
       ...,
       [1.],
       [2.],
       [2.]])

In [33]:
y_pred.shape

(87395, 1)

In [34]:
df_test['Surge_Pricing_Type'] = y_pred

In [35]:
df_test['Surge_Pricing_Type'] = df_test['Surge_Pricing_Type'].astype(int)

In [36]:
submit = df_test[["Trip_ID","Surge_Pricing_Type"]].copy()

In [37]:
submit.to_csv('H:/Learnings/Data Science/Hackathon/Mobility Analytics/sample_submission_4.csv', index=False)

### Using LGBM

In [38]:
lgbm = LGBMClassifier()
lgbm.fit(x_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [39]:
y_pred = lgbm.predict(x_test)

In [40]:
y_pred

array([1, 2, 2, ..., 1, 2, 2], dtype=int64)

In [41]:
y_pred.shape

(87395,)

In [42]:
y_pred = y_pred.reshape(87395,1)

In [43]:
df_test['Surge_Pricing_Type'] = y_pred

In [44]:
submit = df_test[["Trip_ID","Surge_Pricing_Type"]].copy()

In [45]:
submit.to_csv('H:/Learnings/Data Science/Hackathon/Mobility Analytics/sample_submission_5.csv', index=False)